# Imports

In [11]:
from nnet.model import Sequential
from nnet.layers import (
    Input,
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
)
from nnet.utils import one_hot
import numpy as np
import os
from scipy.io import loadmat
import matplotlib.pyplot as plt

# Loading Data

In [2]:
DATA_DIR = "data"
X_train = loadmat(os.path.join(DATA_DIR, "train_images.mat"))["train_images"]
X_test = loadmat(os.path.join(DATA_DIR, "test_images.mat"))["test_images"]
y_train = loadmat(os.path.join(DATA_DIR, "train_labels.mat"))["train_labels"]
y_test = loadmat(os.path.join(DATA_DIR, "test_labels.mat"))["test_labels"]


## Preprocessing

In [3]:
# Making the data compatible with the model
X_train = X_train.T
X_test = X_test.T
X_train = X_train[:, :, np.newaxis, :]
X_test = X_test[:, :, np.newaxis, :]
X_train.shape, X_test.shape

((28, 28, 1, 1000), (28, 28, 1, 1000))

In [4]:
# Normalizing the data
X_train = X_train / X_train.max() - 0.5
X_test = X_test / X_test.max() - 0.5

In [5]:
# One hot encoding the labels
y_train = np.squeeze(y_train.T)
y_test = np.squeeze(y_test.T)
y_train = one_hot(y_train, 10)
y_test = one_hot(y_test, 10)

y_train.shape, y_test.shape

((10, 1000), (10, 1000))

# Problems

## Problem 1

> **Note:** I'm using the updated version of the module which I created for the previous assignment. To see how to work with the module, please refer to the previous assignment.

Here is the model:

In [6]:
model = Sequential("CNN Model")
inp = Input((28, 28, 1), name="Input")
conv = Conv2D(9, (3, 3), padding="valid", activation="relu", name="Conv")
pool = MaxPool2D((2, 2), 2, name="MaxPool")
flat = Flatten(name="Flatten")
dense = Dense(10, name="Output", activation="softmax")

model.add(inp)
model.add(conv)
model.add(pool)
model.add(flat)
model.add(dense)

model.summary()

Model: CNN Model
_________________________________________________________________________________________________
╒═════════╤════════════════╤═════════════════╤═════════════════╤═══════════════╤════════════════╕
│ Name    │ Input Shapes   │ Output Shapes   │ Weight Shapes   │ Bias Shapes   │   # Parameters │
╞═════════╪════════════════╪═════════════════╪═════════════════╪═══════════════╪════════════════╡
│ Input   │ (28, 28, 1)    │ (28, 28, 1)     │ None            │ None          │              0 │
├─────────┼────────────────┼─────────────────┼─────────────────┼───────────────┼────────────────┤
│ Conv    │ (28, 28, 1)    │ (13, 13, 9)     │ (3, 3, 1, 9)    │ (9, 1)        │             90 │
├─────────┼────────────────┼─────────────────┼─────────────────┼───────────────┼────────────────┤
│ MaxPool │ (13, 13, 9)    │ (6, 6, 9)       │ None            │ None          │              0 │
├─────────┼────────────────┼─────────────────┼─────────────────┼───────────────┼────────────────┤
│ F

So, the model has just 3340 parameters. Would this be enough to learn the data? Let's find out. For this, we compile the model:

In [7]:
model.compile(
    loss="categorical_cross_entropy",
    metrics=["accuracy"],
    initializer="glorot",
)

Finally, fit the model:

In [8]:
history = model.fit(X_train, y_train, epochs=50, batch_size=32, lr=0.05, verbose=10)

Epoch 0001/0050 | Loss: 2.56050 | Accuracy: 0.18200 | 
Epoch 0002/0050 | Loss: 2.44336 | Accuracy: 0.20800 | 
Epoch 0003/0050 | Loss: 2.35812 | Accuracy: 0.21400 | 
Epoch 0004/0050 | Loss: 2.29177 | Accuracy: 0.21200 | 
Epoch 0005/0050 | Loss: 2.23784 | Accuracy: 0.21100 | 
Epoch 0006/0050 | Loss: 2.19254 | Accuracy: 0.21200 | 
Epoch 0007/0050 | Loss: 2.15412 | Accuracy: 0.30300 | 
Epoch 0008/0050 | Loss: 2.12118 | Accuracy: 0.30900 | 
Epoch 0009/0050 | Loss: 2.09269 | Accuracy: 0.31700 | 
Epoch 0010/0050 | Loss: 2.06748 | Accuracy: 0.32400 | 
Epoch 0011/0050 | Loss: 2.04555 | Accuracy: 0.32700 | 
Epoch 0012/0050 | Loss: 2.02632 | Accuracy: 0.32900 | 
Epoch 0013/0050 | Loss: 2.00957 | Accuracy: 0.33200 | 
Epoch 0014/0050 | Loss: 1.99492 | Accuracy: 0.33000 | 
Epoch 0015/0050 | Loss: 1.98208 | Accuracy: 0.32900 | 
Epoch 0016/0050 | Loss: 1.97072 | Accuracy: 0.33200 | 
Epoch 0017/0050 | Loss: 1.96073 | Accuracy: 0.33300 | 
Epoch 0018/0050 | Loss: 1.95208 | Accuracy: 0.33000 | 
Epoch 0019

The accuracy is very low. This is because the model implemented above is very simple. It has just 3340 parameters. This is not enough to learn the data.

In [9]:
train_acc = model.evaluate(X_train, y_train, metric="accuracy")
print(f"Train Accuracy: {train_acc*100:.2f}%")

Train Accuracy: 36.80%


So, the training accuracy is about 37%. Let's see how the model performs on the test data.

## Problem 2

In [10]:
test_acc = model.evaluate(X_test, y_test, metric="accuracy")
print(f"Test Accuracy: {test_acc*100:.2f}%")

Test Accuracy: 34.30%


Okay, the test accuracy is very close to the training accuracy. This means that the model is not overfitting.